# <center> Aplicação das Regras de Associação <center>

In [21]:
# Importando as bibliotecas necessárias
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules


In [38]:
# Funções utilizadas no código
def processar_valor(valor):
    """ 
    Processa os valores de Produto GPOM e Área de Atuação

    Parâmetros:
    - valor: coluna onde deseja que o 'NÃO INFORMADO' seja retirado

    Retorna:
    - coluna com os novos valores sem o 'NÃO INFORMADO'
    """
    if 'NÃO INFORMADO' in valor:
        return valor.replace(' / NÃO INFORMADO', '')  # Remove apenas a parte 'NÃO INFORMADO'
    else:
        return f'{valor}'

 
def processar_regras_associacao_pf(df_pf, support, confidence):
    """
    Cria um dataframe com as regras de associação para pessoa física.
    
    Parâmetros:
    - df: DataFrame contendo os dados.
    
    Retorna:
    - dataframe com produtos antecedentes, consequentes e parâmetros de avaliação.
    """
    transacao = []

    for cliente_id in df_pf['ID Cliente'].unique():
        # Obtem todas as transações do cliente
        transacoes_cliente = df_pf[df_pf['ID Cliente'] == cliente_id]

        # Agrega transações por produto e data de nascimento
        transacoes_agrupadas = transacoes_cliente.groupby(['Produto GPOM e Área de Atuação', 'Data de Nascimento']).size().reset_index(name='count')

        # Cria uma lista de produtos únicos por cliente
        lista_produtos = list(transacoes_agrupadas['Produto GPOM e Área de Atuação'])

        # Adiciona lista de produtos únicos à lista de transações
        transacao.append(lista_produtos)

    te = TransactionEncoder()
    transacao_te = te.fit(transacao).transform(transacao)

    # Transforma em dataframe
    transacao_transformado = pd.DataFrame(transacao_te, columns=te.columns_)

    items_frequentes_apriori = apriori(transacao_transformado, use_colnames=True, min_support=support)
    items_frequentes_apriori_sorted = items_frequentes_apriori.sort_values(['support'],ascending=False)

    regras_apriori = association_rules(items_frequentes_apriori_sorted, metric='confidence', min_threshold=confidence)
    return regras_apriori

def processar_regras_associacao_pj(df_pj, support, confidence):
    """
    Cria um dataframe com as regras de associação para pessoa jurídica.
    
    Parâmetros:
    - df: DataFrame contendo os dados.
    
    Retorna:
    - dataframe com produtos antecedentes, consequentes e parâmetros de avaliação.
    """

    transacao = []

    for cliente_id in df_pj['ID Cliente'].unique():
        # Obter todas as transações do cliente
        transacoes_cliente = df_pj[df_pj['ID Cliente'] == cliente_id]

        # Agregar transações por produto e data de nascimento
        transacoes_agrupadas = transacoes_cliente.groupby(['Produto GPOM e Área de Atuação']).size().reset_index(name='count')

        # Criar uma lista de produtos únicos por cliente
        lista_produtos = list(transacoes_agrupadas['Produto GPOM e Área de Atuação'])

        # Adicionar lista de produtos únicos à lista de transações
        transacao.append(lista_produtos)

    te = TransactionEncoder() # estanciar função
    transacao_te = te.fit(transacao).transform(transacao)

    # transformar em dataframe
    transacao_transformado = pd.DataFrame(transacao_te, columns=te.columns_)

    items_frequentes_apriori = apriori(transacao_transformado, use_colnames=True, min_support=support)
    items_frequentes_apriori_sorted = items_frequentes_apriori.sort_values(['support'],ascending=False)

    regras_apriori = association_rules(items_frequentes_apriori_sorted, metric='confidence', min_threshold=confidence)
    return regras_apriori

def filter_consequents(regras, antecedent):
    """
    Filtra os consequentes de acordo com um antecedente específico em um DataFrame de regras de associação.
    
    Parâmetros:
    - regras: DataFrame contendo as regras de associação.
    - antecedent: antecedente específico para o qual os consequentes serão filtrados.

    Retorna:
    - Uma lista de produtos consequentes para o antecedente especificado.
    """

    consequents = regras[regras['antecedents'].apply(lambda x: antecedent in x)]['consequents']
    return consequents.explode().unique()

def find_associated_products(client_products, rules_df):
    """
    Gera uma lista de produtos recomendados para um determinado cliente.
    
    Parâmetros:
    - client_products: produtos que o cliente comprou.
    - rules_df: dataframe com as regras de associaçãi

    Retorna:
    - dista de produtos recomendados para um determinado cliente.
    """
    associated_products = set()
    
    # Convertendo client_products para sets para facilitar a comparação
    client_product_sets = [frozenset([item]) for item in client_products]
    
    for client_product_set in client_product_sets:
        for _, row in rules_df.iterrows():
            # Se o produto do cliente é um subconjunto dos antecedentes ou dos consequentes
            if client_product_set.issubset(row['antecedents']) or client_product_set.issubset(row['consequents']):
                # Adicionamos os produtos associados dos antecedentes e dos consequentes
                associated_products.update(row['antecedents'].union(row['consequents']))
    
    client_product_sets_2 =  frozenset(client_products)
    ## Removendo os produtos que o cliente já compra, já que estamos interessados apenas nos novos produtos associados
    associated_products.difference_update(client_product_sets_2)
    
    # Converter o set em uma lista para exibição
    return list(associated_products)



## 1. Carregamento e tratamento das bases de dados

In [23]:
# Importando os dados tratados de pessoa física e jurídica
df_pf = pd.read_csv('../dados/PF_tratados.csv')
df_pj = pd.read_csv('../dados/PJ_tratados.csv')

In [24]:
# Limpeza da coluna Produto GPOM e Área de Atuação
# Pessoa Física
df_pf['Produto GPOM e Área de Atuação'] = df_pf['Área de Atuação e Produto GPOM'].apply(processar_valor)

# Pessoa Jurídica
df_pj['Produto GPOM e Área de Atuação'] = df_pj['Área de Atuação e Produto GPOM'].apply(processar_valor)



## 2. Aplicaçao do Algoritmo Apriori

### 2.1 Pessoa Física

In [25]:
# Filtrar o dataframe
faixas_etarias = {
        '0 a 12 anos': (0, 12),
        '13 a 17 anos': (13, 17),
        '18 a 24 anos': (18, 24),
        '25 a 34 anos': (25, 34),
        '35 a 64 anos': (35, 64),
        '65 anos ou mais': (65, 80),
        'Todas': (0, 80) 
    }
faixas = ['0 a 12 anos', '13 a 17 anos', '18 a 24 anos', '25 a 34 anos', '35 a 64 anos', '65 anos ou mais', 'Todas']
faixa_etaria = '0 a 12 anos'
idade_inicio, idade_fim = faixas_etarias[faixa_etaria]
filtered_df_pf_idade = df_pf[(df_pf['Idade'] >= idade_inicio) & (df_pf['Idade'] <= idade_fim)]

selected_estado = 'PR'
filtered_df_pf_estado = filtered_df_pf_idade[filtered_df_pf_idade['Estado'] == selected_estado]

selected_cidade = 'CURITIBA'
filtered_df_pf = filtered_df_pf_estado[filtered_df_pf_estado['Cidade'] == selected_cidade]
filtered_df_pf.head()

,ID Cliente,Linha de Ação,Produto GPOM,Data de Venda,Classe de Serviço,Data de Nascimento,Gênero,Cidade,Microrregião,Estado,Fim de Vigência,Tipo Pessoa,Área de Atuação,Área de Atuação e Produto GPOM,Idade,Tempo de Contrato,Longitude,Latitude,Produto GPOM e Área de Atuação
23,43969,EDUCACAO,ENSINO FUNDAMENTAL INTERNACIONAL,2022-09-29,ENSINO FUNDAMENTAL - 6º AO 9º ANO,2011-09-05,FEMININO,CURITIBA,CURITIBA,PR,2023-12-15,FÍSICA,NÃO INFORMADO,ENSINO FUNDAMENTAL INTERNACIONAL / NÃO INFORMADO,11,442,-49.264622,-25.419547,ENSINO FUNDAMENTAL INTERNACIONAL
28,11950,EDUCACAO,ENSINO FUNDAMENTAL INTERNACIONAL,2023-11-06,ENSINO FUNDAMENTAL - 6º AO 9º ANO,2012-03-09,MASCULINO,CURITIBA,CURITIBA,PR,2024-12-31,FÍSICA,NÃO INFORMADO,ENSINO FUNDAMENTAL INTERNACIONAL / NÃO INFORMADO,11,421,-49.264622,-25.419547,ENSINO FUNDAMENTAL INTERNACIONAL
29,21866,EDUCACAO,ENSINO FUNDAMENTAL INTERNACIONAL,2022-12-05,ENSINO FUNDAMENTAL - 6º AO 9º ANO,2011-08-08,FEMININO,CURITIBA,CURITIBA,PR,2023-12-31,FÍSICA,NÃO INFORMADO,ENSINO FUNDAMENTAL INTERNACIONAL / NÃO INFORMADO,11,391,-49.264622,-25.419547,ENSINO FUNDAMENTAL INTERNACIONAL
48,88900,EDUCACAO,ENSINO FUNDAMENTAL INTERNACIONAL,2022-01-17,ENSINO FUNDAMENTAL - 6º AO 9º ANO,2011-06-14,FEMININO,CURITIBA,CURITIBA,PR,2022-12-31,FÍSICA,NÃO INFORMADO,ENSINO FUNDAMENTAL INTERNACIONAL / NÃO INFORMADO,10,348,-49.264622,-25.419547,ENSINO FUNDAMENTAL INTERNACIONAL
49,88900,EDUCACAO,ENSINO FUNDAMENTAL INTERNACIONAL,2022-10-25,ENSINO FUNDAMENTAL - 6º AO 9º ANO,2011-06-14,FEMININO,CURITIBA,CURITIBA,PR,2023-12-15,FÍSICA,NÃO INFORMADO,ENSINO FUNDAMENTAL INTERNACIONAL / NÃO INFORMADO,11,416,-49.264622,-25.419547,ENSINO FUNDAMENTAL INTERNACIONAL


In [27]:
# Aplicar o algoritimo apriori usando a função processar_regras_associacao_pf
regras_pf = processar_regras_associacao_pf(filtered_df_pf, 0.01, 0.3)
regras_pf.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(EDUCACAO CONTINUADA),(ENSINO FUNDAMENTAL INTERNACIONAL),0.324089,0.816165,0.215531,0.665037,0.814831,-0.048979,0.548822,-0.251614
1,(EDUCACAO INFANTIL),(EDUCACAO CONTINUADA),0.178288,0.324089,0.081616,0.457778,1.412507,0.023835,1.246558,0.355403
2,(EDUCACAO INFANTIL),(ENSINO FUNDAMENTAL INTERNACIONAL),0.178288,0.816165,0.066561,0.373333,0.457424,-0.078952,0.293354,-0.590754
3,"(EDUCACAO INFANTIL, ENSINO FUNDAMENTAL INTERNA...",(EDUCACAO CONTINUADA),0.066561,0.324089,0.032488,0.488095,1.506054,0.010916,1.320385,0.359973
4,"(EDUCACAO INFANTIL, EDUCACAO CONTINUADA)",(ENSINO FUNDAMENTAL INTERNACIONAL),0.081616,0.816165,0.032488,0.398058,0.487718,-0.034124,0.305404,-0.533519


In [28]:

antecedents = regras_pf['antecedents'].explode().unique()

num_columns = 2

# Dividindo a lista de antecedentes em grupos com base no número de colunas
chunks = [antecedents[i:i+num_columns] for i in range(0, len(antecedents), num_columns)]

print(f'Produtos associados com o perfil de cliente: \n - {faixa_etaria}\n - {selected_estado}\n - {selected_cidade}')
# Iterando sobre cada grupo de antecedentes e exibindo-os
for chunk in chunks:
    for antecedent in chunk:
        consequents = filter_consequents(regras_pf, antecedent)
        print(f"\nProdutos consequentes de {antecedent}:")
        for consequent in consequents:
            print(consequent)

Produtos associados com o perfil de cliente: 
 - 0 a 12 anos
 - PR
 - CURITIBA

Produtos consequentes de EDUCACAO CONTINUADA:
ENSINO FUNDAMENTAL INTERNACIONAL

Produtos consequentes de EDUCACAO INFANTIL:
EDUCACAO CONTINUADA
ENSINO FUNDAMENTAL INTERNACIONAL

Produtos consequentes de ENSINO FUNDAMENTAL INTERNACIONAL:
EDUCACAO CONTINUADA

Produtos consequentes de INICIACAO PROFISSIONAL / TECNOLOGIA DA INFORMAÇÃO - SOFTWARE:
ENSINO FUNDAMENTAL INTERNACIONAL


In [29]:
# Construção do DataFrame para o Treemap
data = {'antecedents': [], 'consequents': [], 'confidence': []}
for index, row in regras_pf.iterrows():
    antecedente = list(row['antecedents'])[0]  # Convertendo frozenset para lista
    consequentes = list(row['consequents'])    # Convertendo frozenset para lista
    confidence = row['confidence']      # Multiplicando por 100 para obter a porcentagem
    for consequent in consequentes:
        data['antecedents'].append(antecedente)
        data['consequents'].append(consequent)
        data['confidence'].append(confidence)

# Criando o DataFrame
df = pd.DataFrame(data)

# Criando uma nova coluna contendo a contagem de ocorrências
df['count'] = 1

# Agrupando os dados para criar as barras empilhadas e ordenando a contagem de antecedentes em ordem decrescente
grouped_df = df.groupby(['antecedents', 'consequents']).size().unstack(fill_value=0)
grouped_df = grouped_df[grouped_df.sum().sort_values(ascending=False).index]

# Criando o gráfico de barras empilhadas
fig = go.Figure()

# Define a paleta de cores em tons de azul
colors = ['royalblue', 'lightblue', 'skyblue', 'deepskyblue', 'dodgerblue', 'cornflowerblue', 'steelblue']

for i, consequent in enumerate(grouped_df.columns):
    fig.add_trace(go.Bar(
        y=grouped_df.index,
        x=grouped_df[consequent],
        name=consequent,
        orientation='h',
        marker=dict(color=colors[i % len(colors)])
    ))

# Atualizando o layout do gráfico
fig.update_layout(
    barmode='stack',
    yaxis_title='Antecedents',
    xaxis_title='Ocorrências',
    title='Gráfico de barras empilhadas relacionando produtos antecedentes com consequentes'
)
#fig.update_layout(height=800, width=1400)

# Exibindo o gráfico
fig.show()

/home/hub-ia/anaconda3/envs/senai-sprint2/lib/python3.12/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/home/hub-ia/anaconda3/envs/senai-sprint2/lib/python3.12/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [30]:
# Recomendação de produtos por cliente
produto_counts = filtered_df_pf.groupby(['ID Cliente', 'Produto GPOM e Área de Atuação']).size()

grupo_clientes = df_pf.groupby(['ID Cliente', 'Data de Nascimento']).size()

# Filtrar para obter apenas clientes com produtos que aparecem mais de uma vez
clientes_com_produtos_repetidos = produto_counts[produto_counts > 2].reset_index()['ID Cliente'].unique()

# Mudar o valor entre [] para mudar o cliente
select_cliente = clientes_com_produtos_repetidos[0]

produtos = filtered_df_pf[filtered_df_pf['ID Cliente'] == select_cliente]['Produto GPOM e Área de Atuação'].unique()
associated_products = find_associated_products(produtos, regras_pf)

print(f'Produtos comprados e recomendados do cliente {select_cliente}\n')
print('Produtos comprados:')
for produto in produtos:
    print(produto)

print('\nProdutos recomendados:')
for recomendado in associated_products:
    print(recomendado)
    

Produtos comprados e recomendados do cliente 1994

Produtos comprados:
EDUCACAO CONTINUADA
ENSINO FUNDAMENTAL INTERNACIONAL

Produtos recomendados:
EDUCACAO INFANTIL
INICIACAO PROFISSIONAL / TECNOLOGIA DA INFORMAÇÃO - SOFTWARE


### 2.1 Pessoa Jurídica

In [35]:
selected_setor_ibge = '01. INDÚSTRIA'
filtered_df_pj_ibge = df_pj[df_pj['Setor IBGE'] == selected_setor_ibge]

selected_porte = 'GRANDE'
filtered_df_pj_porte = filtered_df_pj_ibge[filtered_df_pj_ibge['Porte'] == selected_porte]

selected_estado_pj = 'PR'
filtered_df_pj_estado = filtered_df_pj_porte[filtered_df_pj_porte['Estado'] == selected_estado_pj]

selected_cidade_pj = 'CURITIBA'
filtered_df_pj_cidade = filtered_df_pj_estado[filtered_df_pj_estado['Cidade'] == selected_cidade_pj]

if (selected_porte == 'GRANDE'):
   filtered_df_pj = filtered_df_pj_cidade
else:    
    cnae_options = filtered_df_pj_cidade['Código CNAE'].value_counts().index.to_list()
    selected_cnae = cnae_options[0]
    filtered_df_pj = filtered_df_pj_cidade[filtered_df_pj_cidade['Código CNAE'] == selected_cnae]

In [39]:
# Aplicar o algoritimo apriori usando a função processar_regras_associacao_pf
regras_pj = processar_regras_associacao_pj(filtered_df_pj_cidade, 0.2, 0.5)
regras_pj

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(AUXILIO DIAGNOSTICO),(CONSULTA CLINICA OCUPACIONAL),0.363636,0.318182,0.318182,0.875000,2.750000,0.202479,5.454545,1.000000
1,(CONSULTA CLINICA OCUPACIONAL),(AUXILIO DIAGNOSTICO),0.318182,0.363636,0.318182,1.000000,2.750000,0.202479,inf,0.933333
2,(AUXILIO DIAGNOSTICO),(CONSULTAS POR ESPECIALIDADES),0.363636,0.272727,0.272727,0.750000,2.750000,0.173554,2.909091,1.000000
3,(CONSULTAS POR ESPECIALIDADES),(AUXILIO DIAGNOSTICO),0.272727,0.363636,0.272727,1.000000,2.750000,0.173554,inf,0.875000
4,(APERFEICOAMENTO PROFISSIONAL),(APERFEICOAMENTO PROFISSIONAL / SEGURANÇA DO T...,0.363636,0.500000,0.272727,0.750000,1.500000,0.090909,2.000000,0.523810
5,(APERFEICOAMENTO PROFISSIONAL / SEGURANÇA DO T...,(APERFEICOAMENTO PROFISSIONAL),0.500000,0.363636,0.272727,0.545455,1.500000,0.090909,1.400000,0.666667
6,"(AUXILIO DIAGNOSTICO, CONSULTA CLINICA OCUPACI...",(CONSULTAS POR ESPECIALIDADES),0.318182,0.272727,0.272727,0.857143,3.142857,0.185950,5.090909,1.000000
7,"(AUXILIO DIAGNOSTICO, CONSULTAS POR ESPECIALID...",(CONSULTA CLINICA OCUPACIONAL),0.272727,0.318182,0.272727,1.000000,3.142857,0.185950,inf,0.937500
8,"(CONSULTA CLINICA OCUPACIONAL, CONSULTAS POR E...",(AUXILIO DIAGNOSTICO),0.272727,0.363636,0.272727,1.000000,2.750000,0.173554,inf,0.875000
9,(AUXILIO DIAGNOSTICO),"(CONSULTA CLINICA OCUPACIONAL, CONSULTAS POR E...",0.363636,0.272727,0.272727,0.750000,2.750000,0.173554,2.909091,1.000000


In [40]:
antecedents = regras_pj['antecedents'].explode().unique()

num_columns = 2

# Dividindo a lista de antecedentes em grupos com base no número de colunas
chunks = [antecedents[i:i+num_columns] for i in range(0, len(antecedents), num_columns)]

print(f'Produtos associados com o perfil de cliente: \n - {selected_setor_ibge}\n - {selected_porte}\n - {selected_estado_pj}\n - {selected_cidade_pj}\n - {selected_cnae}')
# Iterando sobre cada grupo de antecedentes e exibindo-os
for chunk in chunks:
    for antecedent in chunk:
        consequents = filter_consequents(regras_pj, antecedent)
        print(f"\nProdutos consequentes de {antecedent}:")
        for consequent in consequents:
            print(consequent)

Produtos associados com o perfil de cliente: 
 - 01. INDÚSTRIA
 - GRANDE
 - PR
 - CURITIBA
 - 1093-7/01

Produtos consequentes de AUXILIO DIAGNOSTICO:
CONSULTA CLINICA OCUPACIONAL
CONSULTAS POR ESPECIALIDADES
APERFEICOAMENTO PROFISSIONAL
NAO SE APLICA

Produtos consequentes de CONSULTA CLINICA OCUPACIONAL:
AUXILIO DIAGNOSTICO
CONSULTAS POR ESPECIALIDADES

Produtos consequentes de CONSULTAS POR ESPECIALIDADES:
AUXILIO DIAGNOSTICO
CONSULTA CLINICA OCUPACIONAL

Produtos consequentes de APERFEICOAMENTO PROFISSIONAL:
APERFEICOAMENTO PROFISSIONAL / SEGURANÇA DO TRABALHO
AUXILIO DIAGNOSTICO
CAPACITACOES RELACIONADAS AS NRS

Produtos consequentes de APERFEICOAMENTO PROFISSIONAL / SEGURANÇA DO TRABALHO:
APERFEICOAMENTO PROFISSIONAL

Produtos consequentes de ACOES CULTURAIS DE DIFUSAO NO ESTADO:
APERFEICOAMENTO PROFISSIONAL / SEGURANÇA DO TRABALHO

Produtos consequentes de METROLOGIA:
APERFEICOAMENTO PROFISSIONAL / SEGURANÇA DO TRABALHO

Produtos consequentes de CAPACITACOES RELACIONADAS AS NRS:

In [41]:
# Construção do DataFrame para o Treemap
data = {'antecedents': [], 'consequents': [], 'confidence': []}
for index, row in regras_pf.iterrows():
    antecedente = list(row['antecedents'])[0]  # Convertendo frozenset para lista
    consequentes = list(row['consequents'])    # Convertendo frozenset para lista
    confidence = row['confidence']      # Multiplicando por 100 para obter a porcentagem
    for consequent in consequentes:
        data['antecedents'].append(antecedente)
        data['consequents'].append(consequent)
        data['confidence'].append(confidence)

# Criando o DataFrame
df = pd.DataFrame(data)

# Criando uma nova coluna contendo a contagem de ocorrências
df['count'] = 1

# Agrupando os dados para criar as barras empilhadas e ordenando a contagem de antecedentes em ordem decrescente
grouped_df = df.groupby(['antecedents', 'consequents']).size().unstack(fill_value=0)
grouped_df = grouped_df[grouped_df.sum().sort_values(ascending=False).index]

# Criando o gráfico de barras empilhadas
fig = go.Figure()

# Define a paleta de cores em tons de azul
colors = ['royalblue', 'lightblue', 'skyblue', 'deepskyblue', 'dodgerblue', 'cornflowerblue', 'steelblue']

for i, consequent in enumerate(grouped_df.columns):
    fig.add_trace(go.Bar(
        y=grouped_df.index,
        x=grouped_df[consequent],
        name=consequent,
        orientation='h',
        marker=dict(color=colors[i % len(colors)])
    ))

# Atualizando o layout do gráfico
fig.update_layout(
    barmode='stack',
    yaxis_title='Antecedents',
    xaxis_title='Ocorrências',
    title='Gráfico de barras empilhadas relacionando produtos antecedentes com consequentes'
)
#fig.update_layout(height=800, width=1400)

# Exibindo o gráfico
fig.show()

/home/hub-ia/anaconda3/envs/senai-sprint2/lib/python3.12/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/home/hub-ia/anaconda3/envs/senai-sprint2/lib/python3.12/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [42]:
# Recomendação de produtos por cliente
produto_counts = filtered_df_pj.groupby(['ID Cliente', 'Produto GPOM e Área de Atuação']).size()

grupo_clientes = df_pj.groupby(['ID Cliente']).size()

# Filtrar para obter apenas clientes com produtos que aparecem mais de uma vez
clientes_com_produtos_repetidos = produto_counts[produto_counts > 2].reset_index()['ID Cliente'].unique()

# Mudar o valor entre [] para mudar o cliente
select_cliente = clientes_com_produtos_repetidos[0]

produtos = filtered_df_pj[filtered_df_pj['ID Cliente'] == select_cliente]['Produto GPOM e Área de Atuação'].unique()
associated_products = find_associated_products(produtos, regras_pj)

print(f'Produtos comprados e recomendados do cliente {select_cliente}\n')
print('Produtos comprados:')
for produto in produtos:
    print(produto)

print('\nProdutos recomendados:')
for recomendado in associated_products:
    print(recomendado)
    

Produtos comprados e recomendados do cliente 028AF07E-525C-E211-BEC9-00155D726C02

Produtos comprados:
ESTAGIO
APERFEICOAMENTO PROFISSIONAL / METALMECÂNICA - FABRICAÇÃO MECÂNICA
APERFEICOAMENTO PROFISSIONAL / GESTÃO
APERFEICOAMENTO PROFISSIONAL / EDUCAÇÃO
EDUCACAO CONTINUADA
ACOES CULTURAIS DE DIFUSAO NO ESTADO
ACOES CULTURAIS NA EDUCACAO
LAUDOS
ANALISE ERGONOMICA DO TRABALHO - NR 17

Produtos recomendados:
APERFEICOAMENTO PROFISSIONAL / SEGURANÇA DO TRABALHO
